In [ ]:
!pip install -r requirements.txt

In [5]:
import pandas as pd
import requests
from datetime import datetime
from tqdm import tqdm, trange
import os
from PIL import Image

In [6]:
from pptx import Presentation
from pptx.util import Inches

# Fotos

0. Tornar quadro público
1. Baixar csv no trello
2. Salvar na mesma pasta do código
3. Alterar o nome do arquivo célula abaixo

In [7]:
arquivo = 'quadro.csv'

### Ler csv

In [8]:
trello = pd.read_csv(arquivo)

### baixar fotos

In [10]:
def get_fotos(tabela, lista):
    data = []
    tabela_lista = tabela.loc[tabela['List Name']==lista]
    for i in trange(1, len(tabela_lista)):
        card = tabela_lista.iloc[i].to_dict()
        card['Board Name'] = card['Board Name'].replace(' ','')
        card['List Name'] = card['List Name'].replace(' ','')
        # print(card)
        path = f'images/{card["Board Name"]}/{card["List Name"]}'
        if not(os.path.exists(path)):
            os.makedirs(path)
        tmp = {'board': card['Board Name'], 'list': card['List Name'], 'card': card['Card Name'], 'description': card['Card Description'], 
                'data': card['Due Date'], 'fotos' : []}
        # print(type(card['Attachment Links']) == type('str'))
        if type(card['Attachment Links']) == type('str'):
            fotos = card['Attachment Links'].split(', ')
            for j in range(0, len(fotos)):
                foto = fotos[j]
                filename = foto.split('/')[-1]
                if filename[-3:] == 'jpg':
                    tmp['fotos'].append(filename)
                    r = requests.get(foto, allow_redirects=True)
                    open(f'images/{card["Board Name"]}/{card["List Name"]}/{filename}', 'wb').write(r.content)
        data.append(tmp)
    return data

4. Altere no nome da lista ('Atividades') mantendo entre parenteses

In [11]:
data = get_fotos(trello, 'Atividades')

100%|██████████| 22/22 [00:50<00:00,  2.32s/it]


### Gerar apresentação

In [18]:
def get_slides():
    prs = Presentation()
    for item in tqdm(data):
        path_report = f'reports/{item["board"]}/{item["list"]}'
        path_img = f'images/{item["board"]}/{item["list"]}'
        if not(os.path.exists(path_report)):
            os.makedirs(path_report)
        
        for foto in item['fotos']:
            # print(foto)
            img_path = f'{path_img}/{foto}'
            img = Image.open(img_path)
            # print(img_path)
            blank_slide_layout = prs.slide_layouts[5]
            slide = prs.slides.add_slide(blank_slide_layout)
            shapes = slide.shapes

            title_shape = shapes.title
            title_shape.text = item["card"]

            left = Inches(0.2)  # 0.93" centers this overall set of shapes
            top = Inches(7)
            width = height = Inches(1.5)

            txBox = slide.shapes.add_textbox(left, top, width, height)
            tf = txBox.text_frame
            date = datetime.strftime(datetime.strptime(data[0]['data'], '%Y-%m-%dT%H:%M:%S.%fZ'),'%d/%m/%Y')
            tf.text = date

            left = Inches(3.5)  # 0.93" centers this overall set of shapes
            top = Inches(1.9)
            width = Inches((img.width/900)*3)
            height = Inches((img.height/1600)*5)

            # pic = slide.shapes.add_picture(img_path, left, top)

            pic = slide.shapes.add_picture(img_path, left, top, height=height, width=width)
            pic.left = (prs.slide_width - pic.width) // 2
            # pic.top = (prs.slide_height - pic.height) // 2
            
    prs.save(f'{path_report}/{item["board"]}-{item["list"]}-{datetime.now()}.pptx')
    return True 


In [19]:
get_slides()

100%|██████████| 22/22 [00:00<00:00, 231.24it/s]


True